# Get Kitten Cattributes

### Slightly Evil

Unfortunately the attributes such as 'dali' or 'otaku' are not listed anywhere in the blockchain. They are calculated from the cat's genes in a black box manner. I would like to decipher that black box but I need data first. So unfortunately I need to scrape the site. To lessen the burden on the the main site I will scrape two other sites that have already themselves scrapped the main site for these attributes. I won't parallelize it and just let it run as long as it takes, its a one time thing. I also just take a small sample, I don't need every cat.

In [398]:
import requests
from time import sleep
import numpy as np
from lxml import html
import pandas as pd
from tqdm import tqdm_notebook
from subprocess import call
import os

In [458]:
#don't download any cats we currently have
df = pd.read_csv('smaller_sample.csv')
cat_db = df.set_index('id',drop=False).to_dict(orient='index')
print(min(cat_db.keys()),max(cat_db.keys()) )

1 9999


In [359]:
def get_cat(cat_id):
    r = requests.get('https://api.cryptokitties.co/kitties/%s'%cat_id)
    r.raise_for_status()
    raw_data = r.json()
    
    keys_i_care_about = ['id', 'name', 'image_url', 'color', 'bio', 'is_fancy','cattributes']

    data ={}

    for k in keys_i_care_about:
        data[k] = raw_data[k]
    
    data['source'] = 'https://api.cryptokitties.co/kitties'
    return data

def get_cat2(cat_id):
    r = requests.get('http://www.kittyexplorer.com/kitties/%s'%cat_id)
    r.raise_for_status()
    
    tree = html.fromstring(r.content)
    
    data={}
    data['id'] = cat_id
    data['image_url']=tree.xpath('/html/body/div/div[1]/div[1]/a/img/@src')[0]
    data['color']= tree.xpath('/html/body/div/div[1]/div[1]/a/img/@style')[0]
    data['name'] = tree.xpath('/html/body/div/div[1]/div[2]/h3/text()')[0]
    data['bio'] = tree.xpath('/html/body/div/div[1]/div[2]/p[2]/text()')[0]
    data['cattributes']= tree.xpath('/html/body/div/div[1]/div[2]/a/button/text()')
    data['source'] = 'http://www.kittyexplorer.com'
    return data

def get_cat3(cat_id):
    r = requests.get('http://cryptokittydex.com/kitties/%s'%cat_id)
    r.raise_for_status()
    
    tree = html.fromstring(r.content)
    
    data={}
    data['name']=tree.xpath('/html/body/div[2]/div/div[2]/h3[1]/text()')[0]
    
    im_url, color = tree.xpath('/html/body/div[2]/div/div[1]/div/div/a/@style')[0].split(';')

    data['image_url'] = im_url[im_url.find('(')+1:im_url.find(')')]
    if data['image_url'][-3:] not in ['svg','png']:
        data['image_url']= None
        
    data['color'] = color
    
    data['cattributes'] = tree.xpath('/html/body/div[2]/div/div[2]/ul[1]/li[8]/a/text()')
    data['id'] = cat_id
   
    data['source'] = 'http://cryptokittydex.com'
    return data

In [360]:
#cat_db={} 

In [459]:
def scrape_cats(cat_db, start_idx=1, end_idx=100, chill_time=1, max_retry=4):
    
    #lets not go in order
    to_get = [i for i in range(start_idx,end_idx) if i not in cat_db]
    print('downloading %d many new cats'%len(to_get))
    for i in tqdm_notebook(np.random.permutation(to_get)):
        if i in cat_db:
            continue

        #simple retry logic
        try_count =0
        failed=True
        while try_count <= max_retry and failed==True:
            try_count+=1
            try:    
                func = np.random.choice([get_cat,get_cat2,get_cat3])
                data = func(i)
                failed = False
                cat_db[i] = data
                #print('*',end='')
            except Exception as e:
                dt = chill_time*try_count**2
                print('x',end='')
                sleep(dt)

        sleep(chill_time)

In [461]:
scrape_cats(cat_db,start_idx=1,end_idx=2,chill_time=.5, max_retry=0)

downloading 0 many new cats


In [438]:
df = pd.DataFrame([ cat_db[k] for k in cat_db ])
#df.sample(n=10)
print(len(df))

In [440]:
df.to_csv('small_sample_2.csv',index=False)

### Grab the images. 

In [368]:
!mkdir images

In [390]:
ids_already_downloaded = [ int(f.split('.')[0]) for f in os.listdir('images') if f[-3:] in ['svg','png']]

In [391]:
images_to_download = list(df[df.id.apply(lambda x: x not in ids_already_downloaded)].sort_values('source').image_url.dropna().values)[:500]
print(len(images_to_download))

500


In [392]:
commands =['wget','-P','images']+images_to_download

In [393]:
call(commands)

0